<a href="https://colab.research.google.com/github/ZerXXX0/Cloud-classification-with-maxvit/blob/main/CCSN_GCD_Combined_MaxVit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

inaiwt_gcd_dataset_path = kagglehub.dataset_download('inaiwt/gcd-dataset')
inaiwt_ccsn_dataset_path = kagglehub.dataset_download('inaiwt/ccsn-dataset')
inaiwt_gcd_ccsn_combined_path = kagglehub.dataset_download('inaiwt/gcd-ccsn-combined')
inaiwt_gcd_ccsn_new_path = kagglehub.dataset_download('inaiwt/gcd-ccsn-new')
inaiwt_maxvit_ccsn_new_path = kagglehub.notebook_output_download('inaiwt/maxvit-ccsn-new')

print('Data source import complete.')


In [ ]:
from urllib.request import urlopen
import timm
import torch
import zipfile,os
from PIL import Image
from pathlib import Path
from torchvision.datasets import ImageFolder
from torchvision import transforms
from torch.utils.data import DataLoader,Dataset
from sklearn.metrics import accuracy_score
from tqdm import tqdm
import torch.nn as nn
import torch.optim as optim

In [ ]:
train_dir = '/kaggle/input/gcd-ccsn-new/GCD CCSN Combined with val/train'
test_dir =  '/kaggle/input/gcd-ccsn-new/GCD CCSN Combined with val/test'
val_dir =  '/kaggle/input/gcd-ccsn-new/GCD CCSN Combined with val/val'

In [ ]:
model = timm.create_model(
    'maxvit_tiny_tf_224.in1k',
    pretrained=False,
    num_classes=7,
)
if torch.cuda.device_count() > 1:
    model = nn.DataParallel(model)

model = model.eval()

# get model specific transforms (normalization, resize)
data_config = timm.data.resolve_model_data_config(model)
trans = timm.data.create_transform(**data_config, is_training=False)

class CustomDataset(Dataset):
    def __init__(self, data_dir, transform=None):
        self.data_dir = data_dir
        self.transform = transform
        self.images = []
        self.labels = []
        self.class_names = os.listdir(data_dir)

        for label, class_name in enumerate(self.class_names):
            class_dir = os.path.join(data_dir, class_name)
            for img_name in os.listdir(class_dir):
                img_path = os.path.join(class_dir, img_name)
                self.images.append(img_path)
                self.labels.append(label)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = self.images[idx]
        image = Image.open(img_path).convert('RGB')
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label  # Ensure this returns a tuple of (image, label)

transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224
    transforms.RandomHorizontalFlip(p=0.5),           # Flip horizontally (clouds are often symmetrical)
    transforms.RandomVerticalFlip(p=0.3),             # Flip vertically (useful for aerial/satellite views)
    transforms.RandomRotation(30),                    # Random rotations to simulate viewpoint changes
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),  # Randomly crop to add variability
    transforms.RandomAutocontrast(),                  # Auto-contrast to normalize lighting
    transforms.RandomAdjustSharpness(sharpness_factor=2),  # Enhance sharpness for fine details
    transforms.ToTensor(),           # Convert images to PyTorch tensors
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.229, 0.224, 0.225])  # Normalize
])


# Create an instance of the CustomDataset
dataset = CustomDataset(data_dir=train_dir, transform=transform)

train_loader = DataLoader(dataset, batch_size=32, shuffle=True)
for data in train_loader:
    print(data[0].shape)  # This will show you the structure of the data being returned
    inputs, targets = data  # Unpack only if it has the correct structure
    break


import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
import time
import csv

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=0.0001, weight_decay=0.0001)

#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("cuda")
model.to(device)
num_epochs = 5

#New with csv added
csv_file = "train_log_tiny1.csv"
with open(csv_file, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Epoch", "Train Loss", "Train Accuracy", "Time (seconds)"])

# Training loop
model.train()  # Set the model to training mode
for epoch in range(num_epochs):
    start_time = time.time()
    correct = 0
    total = 0
    running_loss = 0.0

    progress_bar = tqdm(train_loader, desc=f"Epoch [{epoch+1}/{num_epochs}]")

    for inputs, targets in progress_bar:
        inputs, targets = inputs.to(device), targets.to(device)  # Move to device

        optimizer.zero_grad()  # Zero the gradients
        outputs = model(inputs)  # Forward pass
        loss = criterion(outputs, targets)  # Compute loss
        loss.backward()  # Backward pass
        optimizer.step()  # Update weights

         # Calculate metrics
        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += targets.size(0)
        correct += (predicted == targets).sum().item()

    # Calculate epoch metrics
    epoch_loss = running_loss / len(train_loader)
    epoch_acc = correct / total * 100
    epoch_time = time.time() - start_time

    # Log results to CSV
    with open(csv_file, mode='a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow([epoch + 1, epoch_loss, epoch_acc, epoch_time])

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.2f}%, Time: {epoch_time:.2f}s")

torch.save(model.state_dict(), "model_tiny_GCD1.pth")

torch.Size([32, 3, 224, 224])


Epoch [1/5]: 100%|██████████| 297/297 [03:57<00:00,  1.25it/s]


Epoch [1/5], Loss: 0.9746, Accuracy: 61.98%, Time: 237.85s


Epoch [2/5]: 100%|██████████| 297/297 [03:38<00:00,  1.36it/s]


Epoch [2/5], Loss: 0.7134, Accuracy: 72.15%, Time: 218.79s


...

Epoch [3/5], Loss: 0.6226, Accuracy: 75.53%, Time: 224.48s


Epoch [4/5]: 100%|██████████| 297/297 [03:39<00:00,  1.35it/s]


Epoch [4/5], Loss: 0.5647, Accuracy: 77.93%, Time: 219.73s


Epoch [5/5]: 100%|██████████| 297/297 [03:43<00:00,  1.33it/s]

Epoch [5/5], Loss: 0.5067, Accuracy: 80.21%, Time: 223.67s


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, classification_report

test_dataset = ImageFolder(root=test_dir, transform = transform)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=True)

# 4. Define testing loop
correct_predictions = 0
total_samples = 0
all_targets = []
all_preds = []


# Perform forward pass
#outputs = model(inputs)

# Get predicted class indices
#_, preds = torch.max(outputs, dim=1)

# Compare predictions with ground truth
#correct_predictions += (preds == targets).sum().item()
#total_samples += targets.size(0)

# Store predictions and labels for metrics
#all_targets.extend(targets.tolist())
#all_preds.extend(preds.tolist())

model.eval()

with torch.no_grad():
    for inputs, targets in test_loader:
        inputs, targets = inputs.to(device), targets.to(device)

        outputs = model(inputs)
        _, preds = torch.max(outputs, dim=1)

        correct_predictions += (preds == targets).sum().item()
        total_samples += targets.size(0)

        all_targets.extend(targets.tolist())
        all_preds.extend(preds.tolist())

accuracy = accuracy_score(all_targets, all_preds)
precision = precision_score(all_targets, all_preds, average=None, zero_division=0)
recall = recall_score(all_targets, all_preds, average=None, zero_division=0)
conf_matrix = confusion_matrix(all_targets, all_preds)
report = classification_report(all_targets, all_preds, zero_division=0)

results_file = f"Tiny test1.txt"
with open(results_file, "w") as f:
    f.write(f"Accuracy: {accuracy * 100:.2f}%\n")
    f.write(f"\nPrecision (per class): {precision}\n")
    f.write(f"\nRecall (per class): {recall}\n")
    f.write(f"\nConfusion Matrix:\n{conf_matrix}\n")
    f.write(f"\nClassification Report:\n{report}\n")

print(f"Accuracy: {accuracy * 100:.2f}%")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(conf_matrix)
print(classification_report)

Accuracy: 6.20%
Precision: [0.09120521 0.03623188 0.03686636 0.0443787  0.09452736 0.02380952
 0.36781609]
Recall: [0.30769231 0.02262443 0.05755396 0.0094399  0.05588235 0.16115702
 0.10543657]
[[  56    1   11   27   22   19   46]
 [  28    5   14   42   82   27   23]
 [  11    1    8   62   41   11    5]
 [   2    0   85   15    2 1482    3]
 [ 142    7   56   49   19   59    8]
 [  91    0   24   61    2   39   25]
 [ 284  124   19   82   33    1   64]]
<function classification_report at 0x7c73320cef80>
